In [71]:
#!pip3 install plotly

In [82]:
import pandas
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [143]:
### raw filename r04c11f04p01-ch2sk30fk1fl1.tiff
ch1="GFP"
ch2="RFP"
construct_names = ["construct SBE4"]
treatment_names = ["DMSO","Gherlin 75nM", "Gherlin 150nM","Gherlin 300nM","TGFbeta 2ng/mL", "TGFbeta 5ng/mL", "TGFbeta 10ng/mL"]
removeoutliers=True

In [144]:
# def colinforlookup(row):
#     colname = row["col"]
#     if colname=="02":
#         return "construct SBE4"
#     if colname=="03":
#         return "SBE4 construct"
#     if colname=="04":
#         return "SBE4 construct"
#     if colname=="05":
#         return "SBE4 construct"
#     if colname=="06":
#         return "SBE4 construct"
    
def rowinforlookup(row):
    rowname = row["row"]
    if rowname=="r01":
        return "DMSO"
    if rowname=="r02":
        return "Gherlin 75nM"
    if rowname=="r03":
        return "Gherlin 150nM"
    if rowname=="r04":
        return "Gherlin 300nM"
    
def channellookup(row):
    microscopechannel = row["colorcode"]
    if microscopechannel.startswith("ch1"):
        return ch1
    if microscopechannel.startswith("ch2"):
        return ch2

# def fixtime(time):
#     if time.find('1fk1fl1-1') == 0:
#         return '1fk1fl1'
#     elif time.find('10fk1fl1-1') == 0:
#         return '10fk1fl1'
#     else:
#         return time

In [167]:
def findoutliers(df):
    df1=df[df["color"]==ch1]
    df2=df[df["color"]==ch2]
    y = df1['MEDIAN']
    removed_outliers = y.between(y.quantile(.1), y.quantile(.90))
    df1 = df1[removed_outliers]
    y2 = df2['MEDIAN']
    removed_outliers = y2.between(y2.quantile(.1), y2.quantile(.90))
    df2 = df2[removed_outliers]
    return df1, df2

In [168]:
def processrawdata(filename="/Users/jewe1055/experiments/exp64/scripts/SBE4_ImageJ.csv"):
    ### raw filename r04c11f04p01-ch2sk30fk1fl1.tiff
    df = pandas.read_csv(filename)
    df[['filename','imagename']] = df.LABEL.str.split(".",expand=True,)
    df[['platepos','microscopeinfo']] = df["filename"].str.split("-",expand=True,n=1)
    df[["colorcode", "time"]]=df["microscopeinfo"].str.split("sk",expand=True,n=1)
    df["color"]=df.apply(lambda row : channellookup(row), axis = 1)
    df[["rowcol", "frame"]]=df["platepos"].str.split("f",expand=True,n=1)
    df[["row", "col"]]=df["rowcol"].str.split("c",expand=True,n=1)
    df["condition_name"] = df.apply(lambda row : rowinforlookup(row), axis = 1)
    df["construct_name"] = df.apply(lambda row : colinforlookup(row), axis = 1)
    df['time']=df.apply(lambda row : fixtime(row['time']), axis = 1)
    df = df[["LABEL", 'AREA', 'MEAN', 'INTDEN', 'MEDIAN', 'RAWINTDEN', "row", "col", "frame", "color", "time", "imagename", "condition_name", "construct_name"]]
    if removeoutliers==True:
        df1,df2 = findoutliers(df)
    else:
        df1=df[df["color"]==ch1]
        df2=df[df["color"]==ch2]
    #throws away anything we don't have RFP and GFP
    mdf = df1.merge(df2, on=["row", "col", "frame","imagename", "AREA", "time", "condition_name", "construct_name"], suffixes=("_"+ch1, "_"+ch2))
    print('mdf data shape ',df.shape)
    #df1 and df2 should be half of samples then df
    print('ch1 data shape ',ch1, df1.shape)
    print('ch2 data shape ',ch2, df2.shape)
    mdf['ratio'] = mdf["MEDIAN_"+ch1]/mdf["MEDIAN_"+ch2]
    mdf_plusmissing = df1.merge(df2, on=["row", "col", "frame","imagename", "AREA", "time", "condition_name", "construct_name"], suffixes=("_"+ch1, "_"+ch2), how="outer")
    print('mdf+missing data shape ', mdf_plusmissing.shape)
    return mdf


In [169]:
#throws away anything we don't have RFP and GFP
mdf = processrawdata()

mdf data shape  (1954, 14)
ch1 data shape  GFP (801, 14)
ch2 data shape  RFP (763, 14)
mdf+missing data shape  (913, 20)


In [170]:
mdf

,LABEL_GFP,AREA,MEAN_GFP,INTDEN_GFP,MEDIAN_GFP,RAWINTDEN_GFP,row,col,frame,color_GFP,...,imagename,condition_name,construct_name,LABEL_RFP,MEAN_RFP,INTDEN_RFP,MEDIAN_RFP,RAWINTDEN_RFP,color_RFP,ratio
0,r01c02f01p01-ch1sk1fk1fl1.tiff:0004-0081,0.000064,553.831,0.035,553,85290,r01,02,01p01,GFP,...,tiff:0004-0081,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0004-0081,489.636,0.031,486,75404,RFP,1.137860
1,r01c02f01p01-ch1sk1fk1fl1.tiff:0011-0346,0.000155,545.472,0.085,545,203461,r01,02,01p01,GFP,...,tiff:0011-0346,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0011-0346,681.627,0.106,685,254247,RFP,0.795620
2,r01c02f01p01-ch1sk1fk1fl1.tiff:0012-0360,0.000170,464.282,0.079,453,189427,r01,02,01p01,GFP,...,tiff:0012-0360,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0012-0360,618.995,0.105,622,252550,RFP,0.728296
3,r01c02f01p01-ch1sk1fk1fl1.tiff:0015-0406,0.000340,468.973,0.160,471,383620,r01,02,01p01,GFP,...,tiff:0015-0406,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0015-0406,555.685,0.189,556,454550,RFP,0.847122
4,r01c02f01p01-ch1sk1fk1fl1.tiff:0017-0550,0.000369,548.068,0.202,553,485588,r01,02,01p01,GFP,...,tiff:0017-0550,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0017-0550,575.968,0.212,576,510308,RFP,0.960069
5,r01c02f01p01-ch1sk1fk1fl1.tiff:0018-0568,0.000175,485.052,0.085,488,204207,r01,02,01p01,GFP,...,tiff:0018-0568,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0018-0568,514.487,0.090,515,216599,RFP,0.947573
6,r01c02f01p01-ch1sk1fk1fl1.tiff:0024-0589,0.000133,473.363,0.063,473,151476,r01,02,01p01,GFP,...,tiff:0024-0589,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0024-0589,486.413,0.065,484,155652,RFP,0.977273
7,r01c02f01p01-ch1sk1fk1fl1.tiff:0033-0636,0.000149,457.824,0.068,450,163901,r01,02,01p01,GFP,...,tiff:0033-0636,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0033-0636,583.173,0.087,573,208776,RFP,0.785340
8,r01c02f01p01-ch1sk1fk1fl1.tiff:0035-1027,0.002000,592.269,1.228,581,2952460,r01,02,01p01,GFP,...,tiff:0035-1027,DMSO,SBE4 construct,r01c02f01p01-ch2sk1fk1fl1-1.tiff:0035-1027,669.683,1.389,640,3338371,RFP,0.907813
9,r01c02f01p01-ch1sk2fk1fl1.tiff:0004-0081,0.000064,542.825,0.035,545,83595,r01,02,01p01,GFP,...,tiff:0004-0081,DMSO,SBE4 construct,r01c02f01p01-ch2sk2fk1fl1.tiff:0004-0081,462.838,0.030,466,71277,RFP,1.169528


In [171]:
mdf["condition_name"].value_counts()

Gherlin 75nM    389
DMSO            279
Name: condition_name, dtype: int64

# Ghrelin

In [172]:
#https://plotly.com/python/violin/


# def graphaparamater(graphof, treatment):
def graphaparamater(graphof):
    whichtreatment = treatment_names[1]
#     print(whichtreatment)
    controltreatment = treatment_names[0]
#     print(controltreatment)
    controlrowdf = mdf[mdf["condition_name"]==controltreatment]
    onerowdf = mdf[mdf["condition_name"]==whichtreatment]

    onetypedf = pandas.concat([controlrowdf, onerowdf])
#     onetypedf = onetypedf[onetypedf['condition_name']==treatment]
#     print(onerowdf)

    fig = go.Figure()
    fig.add_trace(go.Violin(x=onetypedf['time'][ onetypedf['condition_name'] == controltreatment],
                            y=onetypedf[graphof][ onetypedf['condition_name'] == controltreatment],
                            legendgroup=controltreatment, scalegroup=controltreatment, name=controltreatment,
                            side='negative',
                            line_color='green', points='all')
                 )
    fig.add_trace(go.Violin(x=onetypedf['time'][ onetypedf['condition_name'] == whichtreatment ],
                            y=onetypedf[graphof][ onetypedf['condition_name'] == whichtreatment ],
                            legendgroup=whichtreatment, scalegroup=whichtreatment, name=whichtreatment,
                            side='positive',
                            line_color='orange', points='all')
                 )    
    
    
               
    fig.update_traces(meanline_visible=True)
# #     title=graphof+' '+treatment
#     fig.update_layout(violingap=0, violinmode='overlay',title=title)
    fig.update_layout(violingap=0, violinmode='overlay')
    fig.show()


In [173]:
graphof = "MEDIAN_RFP"
graphaparamater(graphof)

# treatment = 'DMSO'
# treatment = 'Gherlin 75nM'
# graphaparamater(graphof,treatment)

In [174]:
graphof = "MEDIAN_GFP"
graphaparamater(graphof)
# treatment = 'DMSO'
# graphaparamater(graphof,treatment)
# treatment = 'gherlin 75nM'
# graphaparamater(graphof,treatment)

In [175]:
graphof = "ratio"
graphaparamater(graphof)
# treatment = 'DMSO'
# graphaparamater(graphof,treatment)
# treatment = 'gherlin 75nM'
# graphaparamater(graphof,treatment)